## System Setup

In [1]:
import pandas as pd
from pandas.io import sql
import sqlite3
import timeit

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
pwd

'/Users/nicholasbeaudoin/Desktop/ShotLink Analytics'

## Clean, Engineer Features, Move to SQL DB

In [4]:
col_names = ['Tour Code', 'TourDescription', 'Year', 'Tourn.#', 'Player#',
       'Course#', 'PermanentTournament#', 'PlayerFirstName',
       'PlayerLastName', 'Round', 'TournamentName', 'CourseName',
       'Hole', 'HoleScore', 'ParValue', 'Yardage', 'Shot',
       'ShotType(S/P/D)', '#ofStrokes', 'FromLocation(Scorer)',
       'FromLocation(Enhanced)', 'To Location(Scorer)',
       'ToLocation(Enhanced)', 'Distance', 'DistancetoPin',
       'IntheHoleFlag', 'AroundtheGreenFlag', '1stPuttFlag',
       'DistancetoHoleaftertheShot', 'Time', 'Lie', 'Elevation',
       'Slope', 'X Coordinate', 'Y Coordinate', 'Z Coordinate',
       'DistancefromCenter', 'DistancefromEdge', 'Date', 'Left/Right',
       'StrokesGained/Baseline', 'StrokesGainedCategory',
       'RecoveryShot']

In [5]:
conn = sqlite3.connect('Shotlink.db')

In [6]:
years = list(range(2003, 2019))

for i in years:

    print('***Working on year {} data***'.format(i))
    
    # Bring in data from local machine into pandas
    df = pd.read_csv("data/tour_{}.TXT".format(i), sep=';', names = col_names, low_memory=False, encoding='latin-1')

    print("Data loaded in dataframe")
    
    # Drop first row because we created new headers
    df.drop(0, inplace=True)

    # Change NaN to zeros
    df.fillna(0, inplace=True)
    
    # Create full player names
    df['PlayerFullName'] = df['PlayerLastName'] + ", " + df['PlayerFirstName']
    
    # Convert to numeric values
    df['Round'] = pd.to_numeric(df['Round'], errors='coerce')
    df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')
    df['Hole'] = pd.to_numeric(df['Hole'], errors='coerce')
    df['HoleScore'] = pd.to_numeric(df['HoleScore'], errors='coerce')
    df['Yardage'] = pd.to_numeric(df['Yardage'], errors='coerce')
    df['Shot'] = pd.to_numeric(df['Shot'], errors='coerce')
    df['#ofStrokes'] = pd.to_numeric(df['#ofStrokes'], errors='coerce')
    df['DistancetoPin'] = pd.to_numeric(df['DistancetoPin'], errors='coerce')
    df['DistancetoHoleaftertheShot'] = pd.to_numeric(df['DistancetoHoleaftertheShot'], errors='coerce')
    df['DistancefromEdge'] = pd.to_numeric(df['DistancefromEdge'], errors='coerce')
    df['StrokesGained/Baseline'] = pd.to_numeric(df['StrokesGained/Baseline'], errors='coerce')
    print('Converted objects')
    
    # Normalize Distances
    df['DistanceYds'] = df['Distance']//36
    df['DistancetoPinYds'] = df['DistancetoPin']//36
    df['DistancetoHoleaftertheShotYds'] = df['DistancetoHoleaftertheShot']//36
    #df['DistancefromCenterYds'] = df['DistancefromCenter']//36
    df['DistancefromEdgeYds'] = df['DistancefromEdge']//36

    df['DistanceFt'] = round(df['Distance']/12, 1)
    df['DistancetoPinFt'] = round(df['DistancetoPin']/12, 1)
    df['DistancetoHoleaftertheShotFt'] = round(df['DistancetoHoleaftertheShot']/12, 1)
    #df['DistancefromCenterFt'] = round(df['DistancefromCenter']/12, 1)
    df['DistancefromEdgeFt'] = round(df['DistancefromEdge']/12, 1)
    print('Distances normalized')
    
    ### Create tournament course yardage
    # Get unique course names and holes
    CourseNameList = list(df['CourseName'].unique())
    CourseHoleList = list(df['Hole'].unique())

    # Dictionary to store {CourseName : Yardage}
    CourseYardage = {}
    
    print('Iterating through courses to grab total yardages:')
    
    for course in CourseNameList:   ## loop 1

        df_CourseName = df.where(df['CourseName'] == course, 0)
        yardage = 0   ## counter

        for hole in CourseHoleList:  ## loop 2
            hole_tot = df_CourseName[df_CourseName['Hole'] == hole]['Yardage'].iloc[0]
            yardage = yardage + hole_tot ## add previous yardage to new hole yardage

        print('\t', course, yardage)

        # Add yardage to dictionary for specified course
        CourseYardage[course] = yardage

    # Dict --> DF
    df_2 = pd.DataFrame(list(CourseYardage.items()), columns=['CourseName', 'CourseYardage'])

    # Merge DFs
    df_new = pd.merge(df, df_2, on = 'CourseName')
    
    print('Writing to SQL DB PGA_Stats table')
    
    # Move to SQL DB
    df_new.to_sql("PGA_Stats".format(i), con=conn, if_exists='append', index=False)
    
    # Delete DF from local memory
    del df
    print('\n')

***Working on year 2003 data***
Data loaded in dataframe
Converted objects
Distances normalized
Iterating through courses to grab total yardages:
	 Plantation Course at Kapalua 7263
	 Waialae CC 7060
	 TPC Scottsdale 7089
	 PGA West (Palmer) 6930
	 La Quinta CC 7060
	 Indian Wells CC 6478
	 Bermuda Dunes CC 6927
	 Poppy Hills 6833
	 Pebble Beach GL 6798
	 Spyglass Hill GC 6862
	 Torrey Pines GC (South) 7568
	 Torrey Pines (North) 6874
	 Riviera CC 7222
	 Tucson National GC 7109
	 Doral CC (Blue) 7125
	 CC at Mirasol (Sunset) 7157
	 Bay Hill Club & Lodge 7239
	 TPC Sawgrass 7093
	 TPC Sugarloaf 7293
	 Harbour Town GL 6973
	 Redstone GC (Fall Creek) 7508
	 English Turn G&CC 7116
	 Quail Hollow CC 7396
	 TPC Four Seasons Resort 7017
	 Cottonwood Valley GC 6846
	 Colonial CC 7080
	 Muirfield Village GC 7265
	 TPC Avenel 6987
	 Westchester CC 6783
	 TPC Southwind 7069
	 Cog Hill G&CC 7320
	 Brown Deer Park GC 6759
	 En-Joie GC 6974
	 TPC River Highlands 6820
	 Warwick Hills G&CC 7127
	 Mont

/Users/nicholasbeaudoin/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)




***Working on year 2004 data***
Data loaded in dataframe
Converted objects
Distances normalized
Iterating through courses to grab total yardages:
	 Plantation Course at Kapalua 7263
	 Waialae CC 7060
	 Bermuda Dunes CC 6962
	 PGA West (Palmer) 6930
	 Indian Wells CC 6478
	 La Quinta CC 7060
	 TPC Scottsdale 7216
	 Pebble Beach GL 6816
	 Spyglass Hill GC 6862
	 Poppy Hills 6833
	 Torrey Pines GC (South) 7568
	 Torrey Pines (North) 6874
	 Riviera CC 7260
	 Tucson National GC 7109
	 Doral CC (Blue) 7266
	 CC at Mirasol (Sunset) 7468
	 Bay Hill Club & Lodge 7267
	 TPC Sawgrass 7093
	 TPC Sugarloaf 7293
	 Harbour Town GL 6973
	 Redstone GC (Fall Creek) 7508
	 English Turn G&CC 7078
	 Quail Hollow Club 7442
	 TPC Four Seasons Resort 7016
	 Cottonwood Valley GC 6847
	 Colonial CC 7054
	 TPC Southwind 7103
	 Muirfield Village GC 7265
	 Westchester CC 6751
	 TPC Avenel 6987
	 Cog Hill G&CC 7326
	 TPC Deere Run 7193
	 En-Joie GC 6974
	 Brown Deer Park GC 6759
	 Warwick Hills G&CC 7127
	 Montre

	 TPC Summerlin 7223
	 Grayhawk GC (Raptor) 7125
	 Palm GC 7010
	 Magnolia GC 7516
Writing to SQL DB PGA_Stats table


***Working on year 2010 data***
Data loaded in dataframe
Converted objects
Distances normalized
Iterating through courses to grab total yardages:
	 Plantation Course at Kapalua 7411
	 Waialae CC 7044
	 La Quinta CC 7060
	 Silver Rock 7403
	 PGA West (Palmer) 6950
	 PGA West (Nicklaus) 6890
	 Torrey Pines GC (South) 7698
	 Torrey Pines (North) 6986
	 Riviera CC 7325
	 Pebble Beach GL 6816
	 Monterey Peninsula CC 6838
	 Spyglass Hill GC 6953
	 TPC Scottsdale 7216
	 PGA National (Champion) 7158
	 TPC Blue Monster at Doral 7334
	 Innisbrook Resort (Copperhead) 7340
	 Bay Hill Club & Lodge 7353
	 Redstone GC (Tournament) 7457
	 Harbour Town GL 6973
	 TPC Louisiana 7399
	 Quail Hollow Club 7469
	 TPC Sawgrass 7215
	 TPC San Antonio 7435
	 TPC Four Seasons Resort 7166
	 Colonial CC 7204
	 Muirfield Village GC 7366
	 TPC Southwind 7239
	 TPC River Highlands 6841
	 Aronimink GC

	 La Quinta CC 7060
	 PGA West (Tournament) 7159
	 Torrey Pines GC (South) 7698
	 Torrey Pines (North) 7052
	 TPC Scottsdale 7266
	 Spyglass Hill GC 6953
	 Monterey Peninsula CC 6867
	 Pebble Beach GL 6816
	 Riviera CC 7322
	 PGA National (Champion) 7140
	 Trump National Doral 7543
	 Innisbrook Resort (Copperhead) 7340
	 Bay Hill Club & Lodge 7419
	 Coco Beach Golf & CC 7506
	 Austin Country Club 7073
	 GC of Houston 7441
	 Harbour Town GL 7099
	 TPC San Antonio - AT&T Oaks 7435
	 TPC Louisiana 7425
	 Quail Hollow Club 7575
	 TPC Sawgrass 7215
	 TPC Four Seasons Resort 7166
	 Colonial CC 7209
	 Muirfield Village GC 7392
	 TPC Southwind 7244
	 Congressional CC (Blue) 7569
	 Firestone CC (South) 7400
	 RTJ Trail (Grand National) 7302
	 Glen Abbey GC 7253
	 Baltusrol GC 7428
	 TPC River Highlands 6841
	 TPC Deere Run 7268
	 Sedgefield CC 7127
	 Bethpage State Park (Black) 7468
	 TPC Boston 7297
	 Crooked Stick GC 7567
	 East Lake GC 7385
Writing to SQL DB PGA_Stats table


***Working on y